In [ ]:
# Import libraries 
import scanpy as sc
import pandas as pd 
import matplotlib.pyplot as plt
import anndata as ad 
import numpy as np
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats


In [ ]:
# Laod count matrixes for samples and Transpose (Rows: Genes, Columns: Cells -> Rows: Cells, Columns: Genes)
wt1 = sc.read_csv("/data/BIOL5177/Assessment/WT1/counts_matrix.csv").T
inf1 = sc.read_csv("/data/BIOL5177/Assessment/Infected1/counts_matrix.csv").T
inf2 = sc.read_csv("/data/BIOL5177/Assessment/Infected2/counts_matrix.csv").T


# Add group columns
wt1.obs["Group"] = "Wildtype"
inf1.obs["Group"] = "Infected"
inf2.obs["Group"] = "Infected" 

# Add sample columns
wt1.obs["Sample"] = "Wildtype1"
inf1.obs["Sample"] = "Infected1"
inf2.obs["Sample"] = "Infected2" 

# Put samples togeter in a list
samples = [wt1, inf1, inf2]  

# Pre-processing 

In [ ]:
# Loop through each sample to perform QC steps  
for sample in samples: 
    sample.var["mt"] = sample.var_names.str.startswith("mt-")                      # Add column for the number of mitocondrial genes
    sc.pp.calculate_qc_metrics(sample, qc_vars=["mt"], log1p=False, inplace=True)  # Calculate QC metrics
    sc.pp.filter_genes(sample, min_cells = 2)                                      # Filter genes that are expressed in less than 2 cells  
    sc.pp.filter_cells(sample, min_genes = 40)                                     # Filter cells that have less than 40 genes expressed


In [ ]:
# Loop thorugh for generating violoin plot 

for sample in samples: 
    sc.pl.violin(
        sample,
        ["total_counts", "n_genes_by_counts" , "pct_counts_mt"],
        jitter=0.4,
        multi_panel=True,
    )

In [ ]:
# Loop through for generating scatter plots 

for sample in samples: 
    fig, axes = plt.subplots(1, 2, figsize=(10,5))                                          # Create subplot
    sc.pl.scatter(sample, x="total_counts", y="pct_counts_mt",ax=axes[0], show=False)       # Total counts vs Mitochondrial genes 
    sc.pl.scatter(sample, x="total_counts", y="n_genes_by_counts", ax=axes[1], show=False)  # Total counts vs the Number of genes   
    

In [ ]:
# Filter out based on parameters 

filtered = []

for sample in samples: 
    
    sample = sample[(sample.obs.n_genes_by_counts > 200) & (sample.obs.n_genes_by_counts < 4200), :]   # Filter cells that have less than 200 genes & have more than 4200 genes
    sample = sample[sample.obs.pct_counts_mt < 5, :].copy()                                            # Filter cells that have more than 5 mitocondrial genes  
   
    filtered.append(sample) 

# Normalizing data

In [ ]:
# Merge all dataset in 'filtered' list 
combined = ad.concat(filtered)
combined.obs
combined_raw = combined.copy()  # Make copy the data befor normalization step                                                      

# delete the unnecesaary objects 
del wt1
del inf1
del inf2

In [ ]:
# Normalize combinded data 
                                                    
sc.pp.normalize_total(combined, target_sum=10e4)                                   # Nomalized the data 
sc.pp.log1p(combined)                                                              # Logarithmize the data
sc.pp.highly_variable_genes(combined, min_mean=0.0125, max_mean=5, min_disp=0.5)   # Identify highly-variable genes 
sc.pl.highly_variable_genes(combined)
combined.raw = combined.copy()                                                     # Store 
combined = combined[:,combined.var.highly_variable]                                # Filter with highly variable genes
sc.pp.scale(combined, max_value=10)                                                # Scale the data 


### Principal Component Analysis (PCA)

In [ ]:
sc.tl.pca(combined, svd_solver="arpack")                 # Reduce the dimensionality of the data
sc.pl.pca_variance_ratio(combined, log=True, n_pcs = 50) # Create elbow plot 

In [ ]:
sc.pl.pca(combined, color="Group")

### Clustering the neighborhood graph

In [ ]:
sc.pp.neighbors(combined, n_neighbors = 15, n_pcs = 32)          # Create the neighborhood graph 
sc.tl.leiden(combined, resolution = 0.2)                         # Cluster with Leiden graph method

sc.tl.umap(combined)                                             # Run UMAP
sc.pl.umap(combined, color = "Group")                            # Plot the clusters 
sc.pl.umap(combined, color = "leiden", legend_loc = "on data")   # To compare cluster's location 

 # Integration with BBKNN 

In [ ]:
# code refererence: https://www.sc-best-practices.org/cellular_structure/integration.html 
combined_b = combined.copy()                           # To avoid to overlap the result from harmony integration
sc.pp.pca(combined_b)                                  # Run PCA 
sc.external.pp.bbknn(combined_b, batch_key = "Group")  # Run BBKNN
sc.pl.pca_variance_ratio(combined_b, log=True, n_pcs = 50)

## Perform analysis

In [ ]:
sc.tl.leiden(combined_b, resolution = 0.2)                          # Cluster with Leiden graph method
sc.tl.umap(combined_b)                                              # Run UMAP
sc.pl.umap(combined_b, color = "Group")                             # Plot the clusters based on "Group"
sc.pl.umap(combined_b, color = "leiden", legend_loc = "on data")    # Plot the clusters based on "Leiden"

## Annotation of Cell type with three largest clusters

In [ ]:
threeclusters = sc.tl.rank_genes_groups(combined_b, "leiden", method="wilcoxon", groups = ["0","1","2"]) # Find marker genes for three largest groups
sc.pl.rank_genes_groups(combined_b, n_genes=25, sharey=False)                                            # Visulalize the top 25 markers                                          

In [ ]:
# Generate a DE table 

res = combined_b.uns["rank_genes_groups"]                   # Extrac the result
groups = res["names"].dtype.names                           # Make a list of cluster's name 
pd.DataFrame(                                               # Construct a data frame with statictics data 
    {
        f"Cluster{group}_{key}": res[key][group]
        for group in groups
        for key in ["names", "pvals_adj","logfoldchanges"]
    })


In [ ]:
# reference: https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html
marker_genes = {
    "CD14+ Mono": ["Cd14"],
    "CD16+ Mono": ["Tcf7l2", "Lyn"],
    "cDC2": ["Cst3", "Cotl1", "Dmxl2", "Clec10a"],
    "Erythroblast": ["Mki67"],
    "Proerythroblast": ["Cdk6", "Syngr1", "Gypa"],
    "NK": ["Nkg7", "Cd247", "Fcer1g", "Tyrobp", "Klrg1"],
    "ILC": ["Id2", "Plcg2", "Syne1"],
    "Naive CD20+ B": ["Ms4a1", "Ighd", "Fcrl1", "Ighm"],
    "B cells": [
        "Ms4a1", "Itgb1", "Col4a4", "Prdm1", "Irf4",
        "Pax5", "Bcl11a", "Blk", "Ighd", "Ighm"
    ],
    "Plasma cells": ["Mzb1", "Hsp90b1", "Fndc3b", "Prdm1", "Igkc", "Jchain"],
    "Plasmablast": ["Xbp1", "Prdm1", "Pax5"],
    "CD4+ T": ["Cd4", "Il7r", "Trbc2"],
    "CD8+ T": ["Cd8a", "Gzma", "Ccl5", "Gzmb"],
    "T naive": ["Lef1", "Ccr7", "Tcf7"],
    "pDC": ["Gzmb", "Il3ra", "Cobll1", "Tcf4"],
}

In [ ]:
sc.pl.dotplot(combined_b, marker_genes, groupby="leiden", use_raw=True) # To compare the expression of marker genes 

In [ ]:
# Annotate cell type for three largest clusters 

combined_b.obs["celltype"] = "None"               
markers = ["B cell", "B cell", "NK cell"]
for i in  range(len(markers)): 
    loc = combined_b.obs["leiden"] == str(i) 
    combined_b.obs.loc[loc,"celltype"] = markers[i]   

combined_b.obs

In [ ]:
largest = combined_b[combined_b.obs["leiden"].isin(["0","1","2"])] # Make subset  for hte largest three clusters 
sc.pl.umap(largest, color=["celltype", "leiden"], groups = ["B cell","NK cell","0","1","2"], legend_loc = "on data")

## Compare frequency between groups

In [ ]:
# Create a table for cell frequency per cluster and visualize as the stacked bar plot
freq_table = pd.crosstab(combined_b.obs["Group"], combined_b.obs["leiden"], normalize="index")
freq_table = freq_table.reindex(["Wildtype","Infected"])

# Visualization -> reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html#pandas.DataFrame.plot.bar
bar_plt = freq_table.plot.bar(stacked=True, colormap="viridis", figsize = (10,6))
plt.legend(title="Clusters", loc = "right")
plt.xticks(rotation=0) 
plt.ylabel("Frequency of cells")

## Differential Expression -> Focusing on "Ms4a1" gene

In [ ]:
# Find the clusters that shows highly expressed Ms4a1 gene
sc.tl.umap(combined_b)                                             # Run UMAP
sc.pl.umap(combined_b, color = "leiden", legend_loc = "on data")   # To compare cluster's location 
sc.pl.umap(combined_b, color = "Ms4a1")                            # To find the clusters that express Ms4a1

In [ ]:
# DE ananlysis with cluster 1 that express Ms4a1 gene highly 
cluster1 = combined_b[combined_b.obs["leiden"]== "1", :].copy()                                                   # Make a subset for cluster 1
sc.tl.rank_genes_groups(cluster1, groupby="Group", groups=["Infected"], reference = "Wildtype",  use_raw = True)  # Run DE in Cluster 1 subset 

res = cluster1.uns["rank_genes_groups"]  # Extract result 

# Create dataframe of result for only infected cells within cluster 1 

c1_infected = pd.DataFrame(
    {
        "Gene": res["names"]["Infected"],
        "adj_p": res["pvals_adj"]["Infected"],
        "logFC": res["logfoldchanges"]["Infected"]
    })

In [ ]:
# Fiilter up-regulated genes
Upc1_infected = c1_infected[
    (c1_infected["adj_p"] < 0.01) &
    (c1_infected["logFC"] > 0.5)]

# print out the number of filtered up-regulated genes
print(f" The number of up-regulated genes in the malaria infected cells of Cluster 1 is BBKNN: {len(Upc1_infected)}")
b_Upnumber = len(Upc1_infected)

## Pseudo bulk DE based on BBKNN result

In [ ]:
# Code refernce for pseudo bulk
# https://pydeseq2.readthedocs.io/en/latest/auto_examples/plot_minimal_pydeseq2_pipeline.html
# https://github.com/mousepixels/sanbomics_scripts/blob/main/pseudobulk_pyDeseq2.ipynb

In [ ]:
# Restore the raw data
pseudo_b = combined_raw.copy() 

# Create a new layer and insert the raw expression matrix
pseudo_b.layers["counts"] = pseudo_b.X.copy()
# Match the data with cluster 1 of bbknn reslut. 
pseudo_b = pseudo_b[combined_b.obs["leiden"]== "1", cluster1.var_names].copy() 

# combined_b.obs["leiden"]== "1", cluster1.var_names


In [ ]:
#Perform "Pseudo-bulk" 
bulk_b = sc.get.aggregate(
    pseudo_b,
    by=["Sample","Group"],
    layer="counts",
    func="sum")

#Retrieve the result from the "sum" layer  
bulk_b_matrix = bulk_b.layers["sum"]

# Convert count matrix to data frame 
bulk_b.X = bulk_b_matrix
counts = pd.DataFrame(bulk_b.X, columns = bulk_b.var_names, index = bulk_b.obs_names)

In [ ]:
# Perform DE analysis
dds_b = DeseqDataSet(
    counts = counts,
    metadata = bulk_b.obs,
    design = "~Group"
)

# Run DESeq2
dds_b.deseq2()

# Perfrom statistical analysis to compare between two groups
stat_b = DeseqStats(dds_b,  contrast=["Group", "Infected", "Wildtype"])
stat_b.summary()

# Make the result to dataframe
res_b = stat_b.results_df 

# Filter the result to extract up-regulated genes
up_b = res_b[
    (res_b["padj"] < 0.01) &
    (res_b["log2FoldChange"] > 0.5)
]

up_b

# Print up-regulated genes in cluster 1 
print(f" The number of up-regulated genes in the malaria infected cells is BBKNN + Pseudo Bulk: {up_b.shape[0]}")

bulk_b_Upnumber = up_b.shape[0]

# Integration with Harmony

In [ ]:
combined_h = combined.copy()  # To avoid to overlap the result with BBKNN integration                         
sc.external.pp.harmony_integrate(combined_h, key = "Group")

In [ ]:
# Find the clusters that shows highly expressed Ms4a1 gene
combined_h.obsm['X_pca'] = combined_h.obsm['X_pca_harmony']               # reference: https://support.parsebiosciences.com/hc/en-us/articles/7704577188500-How-to-analyze-a-1-million-cell-data-set-using-Scanpy-and-Harmony
sc.pp.neighbors(combined_h, n_neighbors = 15)  
sc.tl.leiden(combined_h, resolution = 0.2)                                # Cluster with Leiden graph method

sc.tl.umap(combined_h)                                                    
sc.pl.umap(combined_h, color = "leiden", legend_loc = "on data")          # To compare cluster's location 
sc.pl.umap(combined_h, color = "Ms4a1")                                   # To find the clusters that express Ms4a1

## DE analysis

In [ ]:
# DE ananlysis with cluster 1 that express Ms4a1 gene highly 
cluster1_h = combined_h[combined_h.obs["leiden"]== "1", :].copy()  # Make a subset for cluster 1
sc.tl.rank_genes_groups(cluster1_h, groupby="Group", groups=["Infected"], reference = "Wildtype",  use_raw = True)  # Run DE in Cluster 1 subset 

res = cluster1_h.uns["rank_genes_groups"]  # Extract result 

# Create dataframe of result for only infected cells within cluster 1 
c1_infected = pd.DataFrame(
    {
        "Gene": res["names"]["Infected"],
        "adj_p": res["pvals_adj"]["Infected"],
        "logFC": res["logfoldchanges"]["Infected"]
    })

In [ ]:
# Fiilter up-regulated genes
Upc1_infected = c1_infected[
    (c1_infected["adj_p"] < 0.01) &
    (c1_infected["logFC"] > 0.5)]

# print out the number of filtered up-regulated genes
print(f" The number of up-regulated genes in the malaria infected cells of Cluster 1 is Harmony: {len(Upc1_infected)}")
h_Upnumber = len(Upc1_infected)

## Pseudo bulk DE based on Harmony result

In [ ]:
# Restore the raw data
pseudo_h = combined_raw.copy() 

# Create a new layer and insert the raw expression matrix
pseudo_h.layers["counts"] = pseudo_h.X.copy()

# Match the data with cluster 1 of harmony reslut. 
pseudo_h = pseudo_h[combined_h.obs["leiden"]== "1", cluster1_h.var_names].copy() 


In [ ]:
#Perform "Pseudo-bulk" 
bulk_h = sc.get.aggregate(
    pseudo_h,
    by=["Sample","Group"],
    layer="counts",
    func="sum")

#Retrieve the result from the "sum" layer  
bulk_h_matrix = bulk_h.layers["sum"]

# Convert count matrix to data frame 
bulk_h.X = bulk_h_matrix
counts = pd.DataFrame(bulk_h.X, columns = bulk_h.var_names, index = bulk_h.obs_names)


In [ ]:
# Perform DE analysis
dds_h = DeseqDataSet(
    counts = counts,
    metadata = bulk_h.obs,
    design = "~Group"
)

# Run DESeq2
dds_h.deseq2()

# Perfrom statistical analysis to compare between two groups
stat_h = DeseqStats(dds_h,  contrast=["Group", "Infected", "Wildtype"])
stat_h.summary()

# Make the result to dataframe
res_h = stat_h.results_df 

# Filter the result to extract up-regulated genes
up_h = res_h[
    (res_h["padj"] < 0.01) &
    (res_h["log2FoldChange"] > 0.5)]

up_h

# Print up-regulated genes in cluster 1 
print(f" The number of up-regulated genes in the malaria infected cells is Harmony + Pseudo Bulk: {up_h.shape[0]}")

bulk_h_Upnumber = up_h.shape[0]

## Result bar plot 

In [ ]:
# Summurize the integration and pesudo bulk analysis results 

# Create the dictionay to gather the result 
up_numbers = {"BBNKK": b_Upnumber,
              "BBKNN:Pseudo Bulk" : bulk_b_Upnumber,
              "Harmony": h_Upnumber,
              "Harmony:Pseudo Bulk": bulk_h_Upnumber }

# each bar colors 
colors = ['royalblue', 'cornflowerblue', 'slateblue', 'thistle']

# Create bar color 
plt.bar(range(len(up_numbers)), list(up_numbers.values()), tick_label = list(up_numbers.keys()), color = colors)

# Assignn each value on bar
for i, val in enumerate(list(up_numbers.values())):
    plt.text(i, val + 0.1, int(val), ha='center', va='bottom')
    
plt.title('Up-regulated genes of malaria infected cells')
plt.ylabel('the number of genes')


# Trajectory analysis

## PAGA

In [ ]:
# Preprocessing and clustering 
paga = combined.copy()                           # To avoid to overlap the result
sc.tl.pca(paga, svd_solver="arpack")             # Run PCA 
sc.pp.neighbors(paga, n_neighbors=15, n_pcs=32)  # Compute neighbors 
sc.tl.leiden(paga, resolution = 0.2)             # clustering 

In [ ]:
# Denoising the graph
sc.tl.diffmap(paga)                                         # compute a diffusion map                                  
sc.pp.neighbors(paga, n_neighbors=15, use_rep="X_diffmap")

In [ ]:
# Run PAGA
sc.tl.paga(paga, groups="leiden")           # Run Paga
sc.pl.paga(paga, color=["leiden", "Ms4a1"]) # Focus on Ms4a1 marker gene 

## Embedding on UMAP 

In [ ]:
# Run and visualize UMAP
sc.tl.umap(paga, init_pos = "paga")                                    # Run UMAP
sc.pl.umap(paga, color = ["leiden", "Ms4a1"], legend_loc = "on data")  # Visualize the UMAP result

## Embedding on PHATE

In [ ]:
sc.external.tl.phate(paga, n_pca = 32)                                          # Run Phate
sc.external.pl.phate(paga, color = ["leiden", "Ms4a1"], legend_loc = "on data") # Visulalize Phate result 